In [1]:
import pandas as pd
import numpy as np


%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
sl_df = pd.read_excel("../Data/Service Line Data.xls")
res_df = pd.read_csv("../Data/residential_test_data.csv")

In [3]:
merged_df = pd.merge(res_df, sl_df, left_on='PID Dash', right_on='PIDdash',how='left')

In [4]:

merged_df.rename(columns={'Year Built': 'res_year_built', 'Year_Built': 'sl_year_built'}, inplace=True)


dummy_cols = ['Property Zip Code', 'Owner Type', 'Residential Building Style', 'Homestead',
              'Building Storeys', 'Rental', 'Use Type', 'Prop Class', 'Old Prop class', 'USPS Vacancy',
              'Housing Condition 2012', 'Housing Condition 2014',
              'Commercial Condition 2013', 'Hydrant Type']

cols_to_drop = ['Sample Number','Date Submitted','Analysis (Lead)','Analysis (Copper)',
               'City','Full Address','Best Address','Clean Address','PID Dash',
               'Property Address','Owner Name','Owner Address','Owner Zip Code',
                'Owner City','Owner State','Owner Country','Tax Payer Name','Tax Payer Address',
                'Tax Payer State','Tax Payer Zip Code','Prop Class',
               'goog_address','Longitude_y', 'Latitude_y', 'Prop_City', 'Prop_State', 'Prop_Zip',
               'Prop_Addr','PIDdash']

cols_to_factorize = ['Street Name','Zip Code','Property Zip Code','Owner Type','Homestead',
                    'Residential Building Style','Rental','Use Type','Old Prop class',
                    'USPS Vacancy','Zoning','Future Landuse','DRAFT Zone','Housing Condition 2012',
                    'Housing Condition 2014','Commercial Condition 2013','Hydrant Type','Class',
                    'Street #']

merged_df.drop(cols_to_drop, axis=1, inplace=True)

for i, row in merged_df.iterrows():
    row['Zip Code'] = str(row['Zip Code'])[:5]

merged_df.replace(to_replace=" ",value="",inplace=True)
merged_df.fillna(value=-999,inplace=True)
# factorize string-based features for XGBoost
for (feat_name, feat_series) in merged_df.iteritems():
    if feat_name in cols_to_factorize:
        merged_df[feat_name], tmp_indexer = pd.factorize(merged_df[feat_name])
    print(feat_name)
    merged_df[feat_name] = merged_df[feat_name].astype(float)

Lead (ppb)
Copper (ppb)
Street #
Street Name
Zip Code
PID no Dash
Property Zip Code
Owner Type
Homestead
Homestead Percent
HomeSEV
Land Value
Land Improvements Value
Residential Building Value
Residential Building Style
Commercial Building Value
Building Storeys
Parcel Acres
Rental
Use Type
Old Prop class
res_year_built
USPS Vacancy
Zoning
Future Landuse
DRAFT Zone
Housing Condition 2012
Housing Condition 2014
Commercial Condition 2013
Latitude_x
Longitude_x
Hydrant Type
sl_year_built
Class
Ward
PRECINCT
CENTRACT


ValueError: could not convert string to float: 

In [ ]:
# Now that our data is loaded, clean, and ready to go, create numpy arrays to pass
# to a classifier.

Ydata = (merged_df['Lead (ppb)'] > 15)
Xdata = merged_df.drop(['Lead (ppb)', 'Copper (ppb)'], axis=1).values

In [ ]:
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
from sklearn.cross_validation import StratifiedKFold

from sklearn.metrics import roc_auc_score as roc

class XGBClassifier_new(XGBClassifier):
    def predict(self, X):
        return XGBClassifier.predict_proba(self, X)[:,1]

cross_validation = StratifiedKFold(Ydata, n_folds=3, shuffle=True,random_state=0)

xgtrain = xgb.DMatrix(Xdata,label=Ydata)
xgmodel = XGBClassifier(n_estimators=1000,seed=1,learning_rate=.03,objective='binary:logistic',nthread=-1)
xgb_param = xgmodel.get_xgb_params()
cvresult = xgb.cv(xgb_param,xgtrain, num_boost_round=model.get_params()['n_estimators'],nfold=3,metrics=['auc'],
                 early_stopping_rounds=50, show_progress=True)